In [1]:
%load_ext autoreload
%autoreload 2


In [22]:
from notte.env import NotteEnv
model = "groq/llama-3.3-70b-versatile"
async with NotteEnv(model=model, headless=False, persist=True) as env:
  # observe a webpage, and take a random action
  obs1 = await env.observe("https://www.google.com/travel/flights")
  context1 = env._context
  obs2 = await env.step("I3", "Paris")
  context2 = env._context
  obs3 = await env.step("I4", "Paris")
  context3 = env._context
  #obs = await env.execute("L13")
  # obs = await env.execute("L4")
  # obs = await env.execute("I3", "Paris")
  # obs = await env.execute("I4", "Paris")
# await env._browser.page.get_by_role("link", name="Flights").all()

2024-12-09 15:17:54.914 | INFO     | notte.common.logging:wrapper:16 - Function observe took 9.3964 seconds
2024-12-09 15:17:57.153 | INFO     | notte.pipe.listing:forward_incremental:45 - 🚀 forward incremental
2024-12-09 15:17:57.155 | INFO     | notte.common.logging:wrapper:16 - Function step took 2.2366 seconds
2024-12-09 15:17:59.369 | INFO     | notte.pipe.listing:forward_incremental:45 - 🚀 forward incremental
2024-12-09 15:17:59.370 | INFO     | notte.common.logging:wrapper:16 - Function step took 2.2147 seconds


In [15]:
context1.node.find("I4")

NotteNode(id='I4', role=<NodeRole.COMBOBOX: 'combobox'>, text='Where to? ', subtree_ids=['I4'], children=[], attributes_pre=NodeAttributesPre(modal=None, required=None, description=None, visible=None, selected=None, checked=None, enabled=None, path='https://www.google.com/travel/flights:WebArea:Google Flights - Find Cheap Flight Options & Track Prices:group::group::search:Flight:group::group::group::combobox:Where to? '), attributes_post=None)

In [20]:
from notte.pipe.resolution import ActionNodeResolutionPipe 
from notte.actions.base import Action
await ActionNodeResolutionPipe(env._browser).forward(Action(id="I4", description="", category="input"), [], context2)

ExecutableAction(id='I4', description='', category='input', params=[], status='valid', node=NotteNode(id='I4', role=<NodeRole.COMBOBOX: 'combobox'>, text='Where to? ', subtree_ids=['I4'], children=[], attributes_pre=NodeAttributesPre(modal=None, required=None, description=None, visible=None, selected=None, checked=None, enabled=None, path='https://www.google.com/travel/flights?tfs=CBwQARoOagwIAxIIL20vMDVxdGoaDnIMCAMSCC9tLzA1cXRqQAFIAXABggELCP___________wGYAQE&tfu=KgIIAw:WebArea:Google Flights - Find Cheap Flight Options & Track Prices:group::group::search:Flight:group::group::group::combobox:Where to? '), attributes_post=NotteAttributesPost(selectors=HtmlSelector(playwright_selector='internal:role=combobox[name="Where to? "i] >> nth=0 >> nth=0', css_selector='div#i23 > div:nth-of-type(4) > div > div > div > div > div > input', xpath_selector='//*[@id="i23"]/div[4]/div[1]/div[1]/div[1]/div[1]/div[1]/input[1]'), editable=True, input_type='text', text='')), params_values=[], code=None)

2024-12-09 15:00:13.355 | INFO     | notte.pipe.listing:forward_incremental:45 - 🚀 forward incremental
2024-12-09 15:00:13.356 | INFO     | notte.common.logging:wrapper:16 - Function step took 2.2755 seconds


In [8]:

env._context.node.find("I4")


NotteNode(id='I4', role=<NodeRole.COMBOBOX: 'combobox'>, text='Where to? ', subtree_ids=['I4'], children=[], attributes_pre=NodeAttributesPre(modal=None, required=None, description=None, visible=None, selected=None, checked=None, enabled=None, path='https://www.google.com/travel/flights?tfs=CBwQARoOagwIAxIIL20vMDVxdGoaDnIMCAMSCC9tLzA1cXRqQAFIAXABggELCP___________wGYAQE&tfu=KgIIAw:WebArea:Google Flights - Find Cheap Flight Options & Track Prices:group::group::search:Flight:group::group::group::combobox:Where to? '), attributes_post=None)

In [12]:
env._context.subgraph_without(env._action_space.actions(status=["valid", "failed", "excluded"]))


ValueError: No nodes left after filtering of exesting actions

In [3]:
env._action_space

ActionSpace(_actions=[Action(id='L15', description='Opens the Orlando page', category='Navigation', params=[], status='valid'), Action(id='L51', description='Opens the Privacy page', category='Legal & Policy', params=[], status='valid'), Action(id='L3', description='Opens the Explore page', category='Navigation', params=[], status='valid'), Action(id='I2', description='Selects the seating class', category='Search & Input', params=[ActionParameter(name='seatingClass', type='str', default='economy', values=['economy', 'premium economy', 'business', 'first class'])], status='valid'), Action(id='L37', description='Opens the Flights from London to Tokyo page', category='Search & Input', params=[], status='valid'), Action(id='B23', description='Shows answer to frequently asked question', category='Help & Support', params=[], status='valid'), Action(id='B12', description='Selects Sacramento as the origin', category='Search & Input', params=[], status='valid'), Action(id='B11', description='Se

In [ ]:
from notte.env import NotteEnv
import litellm

goal = "Find best flights from Boston to Tokyo"
messages = [{"role": "system", "content": goal}]

def llm_agent(messages):
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=messages,
    )
    return response.choices[0].message.content

async with NotteEnv(headless=False) as env:
  while '<done>' not in messages[-1]['content']:
    resp = llm_agent(messages) # query your llm policy.
    obs = await env.chat(resp) # query notte with llm response.
    messages.append({"role": "assistant", "content": resp})
    messages.append({"role": "user", "content": obs})